## Our New Tversky-Kahneman Loss

In [ ]:
def tversky_kahneman(target, output, gamma=0.279, smooth=1e-10):
  #output = tf.expand_dims(tf.argmax(output, axis=-1), axis = -1)
  #output = tf.cast(output, tf.float32)
  target = tf.cast(target, tf.float32)
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  p = (true_pos + true_neg)/(true_pos + true_neg + false_pos + false_neg)   ###########
  p_gamma = K.pow(p,gamma) #p^gamma
  _p_gamma = K.pow(1-p, gamma) #(1-p)^gamma
  loss = _p_gamma/K.pow(p_gamma + _p_gamma, 1/gamma)

  #print('tversky-kahneman: ', loss)

  return loss

## Traditional Pix2Pix generator loss and discriminator loss

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = tf.compat.v1.losses.sigmoid_cross_entropy(multi_class_labels = tf.ones_like(disc_real_output), logits = disc_real_output)
  generated_loss = tf.compat.v1.losses.sigmoid_cross_entropy(multi_class_labels = tf.zeros_like(disc_generated_output), logits = disc_generated_output)
  total_disc_loss = real_loss + generated_loss

  #print('real_loss: ', real_loss)
  #print('generated_loss: ', generated_loss)

  return total_disc_loss

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = tf.compat.v1.losses.sigmoid_cross_entropy(multi_class_labels = tf.ones_like(disc_generated_output), logits = disc_generated_output) 
  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
  total_gen_loss = 0.01*gan_loss + l1_loss
  #print('gan loss: ', 0.01*gan_loss)
  #print('l1: ', l1_loss)

  return total_gen_loss, gan_loss, l1_loss

## Levelset Loss and Active Contour Loss

In [ ]:
def levelsetLoss(target, output):
  loss = 0.0
  for ich in range(target.shape[3]):
    _target = target[...,ich:ich+1]    
    pcentroid = tf.reduce_sum(_target*output, (1,2), keepdims=True) / tf.reduce_sum(output, (1,2), keepdims=True)   
    plevel = _target - pcentroid
    pLoss = plevel*plevel*output
    loss += tf.reduce_sum(pLoss)
  return loss

def gradientLoss(input, penalty="l1"):
  dH = tf.math.abs(input[:, 1:, :, :] - input[:, :-1, :, :])
  dW = tf.math.abs(input[:, :, 1:, :] - input[:, :, :-1, :])
  if penalty == "l2":
    dH = dH * dH
    dW = dW * dW

  loss =  tf.reduce_sum(dH) +  tf.reduce_sum(dW)
  return loss

In [ ]:
def pre_Active_Contour_Loss(target, output, outDim=2, smooth=0.001):  
  yTrueOnehot = tf.one_hot(tf.squeeze(tf.cast(target, tf.uint8), axis=-1), depth=outDim)
  #yTrueOnehot = tf.broadcast_to(yTrueOnehot, [1,256,256,1])
  #y_pred = tf.broadcast_to(output, [1,256,256,1])
  loss =  output*(1-yTrueOnehot) + (1-output)*yTrueOnehot
  return tf.reduce_mean(loss)

In [ ]:
def Active_Contour_loss(input, target, output, alpha = 1e-7, beta = 1e-2):
  # print('\ractive loss: ',self.activeContourLoss(trueLabel, output),' leverset loss: ', self.levelsetLoss(image,output),'gradient loss: ', self.gradientLoss(output), end="")
  #print('pre-active contour: ', pre_Active_Contour_Loss(target, output))
  loss = alpha*(levelsetLoss(input, output) + beta*gradientLoss(output)) + pre_Active_Contour_Loss(target, output)
  #print('active contour: ', loss)
  return loss

## Dice Coefficient and Jaccard

In [ ]:
def dice_coef(target, output, smooth=1e-10):
  #Average dice coefficient per batch
  axes = (1,2,3)
  intersection = K.sum(target*output, axis=axes)
  summation = K.sum(target+output, axis=axes)
  dice = K.mean((2.0*intersection+smooth)/(summation+smooth), axis=0)
  return dice

def jaccard_coef(target, output, smooth=1e-10):
  #Average jaccard coefficient per batch
  axes = (1,2,3)
  intersection = K.sum(target*output, axis=axes)
  union = K.sum(target+output, axis=axes) - intersection
  jaccard = K.mean((intersection+smooth)/(union+smooth), axis=0)
  return jaccard

def metrics(target, output):
  output_standard = tf.expand_dims(tf.argmax(output, axis=-1), axis = -1)     ########
  _output =  tf.where(output_standard == 1, 1.0, 0.0)    ############ 
  _target =  tf.where(target == 1, 1.0, 0.0)
  #output = tf.broadcast_to(output, [1,256,256,1])
  #target = tf.broadcast_to(target, [1,256,256,1])

  _dice_coef = dice_coef(_target, _output, smooth=1e-10)
  _jaccard_coef = jaccard_coef(_target, _output, smooth=1e-10)
  return _dice_coef, _jaccard_coef

## MVN

In [ ]:
def mvn(tensor, eps=1e-6):
  '''Performs per-channel spatial mean-variance normalization.'''
  mean = K.mean(tensor, axis=(1,2), keepdims=True)
  std = K.std(tensor, axis=(1,2), keepdims=True)
  mvn = (tensor - mean)/(std + eps)

## Precision and Recall

In [ ]:
def confusion(target, output, smooth=1e-10):
  output_positive = K.clip(output, 0, 1)
  output_negative = 1 - output_positive
  target_positive = K.clip(target, 0, 1)
  target_negative = 1 - target_positive
  TP = K.sum(target_positive * output_positive)
  FP = K.sum(target_negative * output_positive)
  FN = K.sum(target_positive * output_negative) 
  precision = (TP + smooth)/(TP + FP + smooth)
  recall = (TP + smooth)/(TP + FN + smooth)
  return precision, recall

In [ ]:
def TP(target, output, smooth=1e-10):
  output_positive = K.round(K.clip(output, 0, 1))
  target_positive = K.round(K.clip(target, 0, 1))
  TP = (K.sum(target_positive * output_positive) + smooth)/ (K.sum(target_positive) + smooth) 
  return TP

def TN(target, output, smooth=1e-10):
  smooth = 1
  output_positive = K.round(K.clip(output, 0, 1))
  output_negative = 1 - output_positive
  target_positive = K.clip(target, 0, 1)
  target_negative = 1 - target_positive
  TN = (K.sum(target_negative * output_negative) + smooth) / (K.sum(target_negative) + smooth)
  return TN

## Tversky Loss

In [ ]:
def tversky(target, output, smooth=1e-10):
  y_true_pos = K.flatten(target)
  y_pred_pos = K.flatten(output)

  true_pos = K.sum(y_true_pos * y_pred_pos)
  false_neg = K.sum(y_true_pos * (1-y_pred_pos))
  false_pos = K.sum((1-y_true_pos)*y_pred_pos)
  alpha = 0.7

  loss = (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

  return loss

def tversky_loss(target, output, smooth=1e-10):
  return 1 - tversky(target, output, smooth=1e-10)

def focal_tversky(target, output, smooth=1e-10):
  p = tversky(target, output, smooth=1e-10)
  gamma = 0.75
  
  return K.pow((1-p), gamma)

## Accuracy

In [ ]:
def accuracy(target, output, smooth=1e-10):
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  p = (true_pos + true_neg + smooth)/(true_pos + true_neg + false_pos + false_neg + smooth)
  return p

## SEN - SPE

In [ ]:
def sen_spe(target, output, smooth=1e-10):
  target_positive = K.flatten(target)
  output_positive = K.flatten(output)

  true_pos = K.sum(target_positive * output_positive)
  true_neg = K.sum((1-target_positive) * (1-output_positive))
  false_neg = K.sum(target_positive * (1-output_positive))
  false_pos = K.sum((1-target_positive) * output_positive)

  sen = true_pos/(true_pos + false_neg)
  spe = true_neg/(true_neg + false_pos)
  return sen, spe

## Cross-Entropy

In [ ]:
def weighted_cross_entropy(target, output, beta):
  output_standard = tf.expand_dims(tf.argmax(output, axis=-1), axis = -1)     ########
  _output =  tf.where(output_standard == 1, 1.0, 0.0)    ############ 
  _target =  tf.where(target == 1, 1.0, 0.0)

  weight_a = beta * tf.cast(_target, tf.float32)
  weight_b = 1 - tf.cast(_target, tf.float32)
    
  o = (tf.math.log1p(tf.exp(-tf.abs(_output))) + tf.nn.relu(-_output)) * (weight_a + weight_b) + _output * weight_b 
  loss = tf.reduce_mean(o)

  return loss

In [ ]:
def balanced_cross_entropy(y_true, y_pred, beta):
  weight_a = beta * tf.cast(y_true, tf.float32)
  weight_b = (1 - beta) * tf.cast(1 - y_true, tf.float32)
    
  o = (tf.math.log1p(tf.exp(-tf.abs(y_pred))) + tf.nn.relu(-y_pred)) * (weight_a + weight_b) + y_pred * weight_b
  loss = tf.reduce_mean(o)

  return loss

In [ ]:
def binary_cross_entropy(y_true, y_pred):
  pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.zeros_like(y_pred))
  pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.ones_like(y_pred))
  loss = K.binary_crossentropy(y_true, y_pred)*pt_1 + K.binary_crossentropy(y_true, y_pred)*pt_0

  return loss